In [44]:
from src.utils import get_files
from src.eval import create_labels_dict, merge_labels, calculate_metrics, calculate_overall_metrics

In [45]:
# params
tag = "dev"

In [46]:
files_labels_llm = get_files(f"./data/labels_llm/{tag}/")
files_labels = get_files(f"./data/labels/")

In [47]:
labels_llm = create_labels_dict(files_labels_llm, tag='dev')
labels = create_labels_dict(files_labels)

In [48]:
labels_final = merge_labels(labels, labels_llm)

Number of common labels: 28


In [49]:
# Calculate metrics
entity_metrics = calculate_metrics(labels_final)
overall_metrics = calculate_overall_metrics(entity_metrics)

In [50]:
overall_metrics

{'Overall Precision': 0.359504132231405,
 'Overall Recall': 0.34523809523809523,
 'Overall F1 Score': 0.3522267206477733}

In [51]:
entity_metrics

defaultdict(<function src.eval.calculate_metrics.<locals>.<lambda>()>,
            {'names': {'TP': 82,
              'FP': 94,
              'FN': 59,
              'Precision': 0.4659090909090909,
              'Recall': 0.5815602836879432,
              'F1 Score': 0.5173501577287066},
             'phone_numbers': {'TP': 1,
              'FP': 1,
              'FN': 28,
              'Precision': 0.5,
              'Recall': 0.034482758620689655,
              'F1 Score': 0.06451612903225806},
             'email_addresses': {'TP': 4,
              'FP': 59,
              'FN': 50,
              'Precision': 0.06349206349206349,
              'Recall': 0.07407407407407407,
              'F1 Score': 0.06837606837606837},
             'physical_addresses': {'TP': 0,
              'FP': 1,
              'FN': 28,
              'Precision': 0.0,
              'Recall': 0.0,
              'F1 Score': 0}})